In [118]:
import os
import pandas as pd
from tqdm.auto import tqdm
import json
import json_repair
from typing import Dict, List, Optional

import tiktoken

from langchain.docstore.document import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

from langchain.text_splitter import RecursiveCharacterTextSplitter

In [136]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
VSE_GPT_API_KEY = ""
os.environ["OPENAI_API_KEY"] = VSE_GPT_API_KEY

# Data

In [84]:
adv_df = pd.read_csv("/Users/alfa/Code/financial_assistant/data/interim/alfa_invest_advanced_paragraphs.csv")
beg_df = pd.read_csv("/Users/alfa/Code/financial_assistant/data/interim/alfa_invest_begginer_paragraphs.csv")

In [85]:
adv_df.info(), beg_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   paragraph_id  29 non-null     object
 1   article_id    29 non-null     object
 2   heading       29 non-null     object
 3   text          29 non-null     object
dtypes: object(4)
memory usage: 1.0+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   paragraph_id  38 non-null     object
 1   article_id    38 non-null     object
 2   heading       38 non-null     object
 3   text          38 non-null     object
dtypes: object(4)
memory usage: 1.3+ KB


(None, None)

In [5]:
adv_df.head()

,chunk_id,article_id,heading,text
0,AIA_0000_0001,AIA_0000,# **Иностранные и российские акции**,"**Акция** — это ценная бумага, позволяющая инв..."
1,AIA_0000_0002,AIA_0000,# **Листинг ценных бумаг**,"Компании, чьи акции доступны неограниченному к..."
2,AIA_0000_0003,AIA_0000,"# **Акции, включённые в индекс**",**Индексы** — это важные индикаторы ситуации н...
3,AIA_0000_0004,AIA_0000,"# **Риски, связанные с покупкой акций иностран...",Иностранные компании и иностранные биржи подчи...
4,AIA_0001_0005,AIA_0001,Облигации с низким кредитным рейтингом,"**Облигация** — это долговая ценная бумага, об..."


In [6]:
beg_df.head()

,chunk_id,article_id,heading,text
0,AIB_0000_0001,AIB_0000,Что такое инвестиции?,"Деньги нужны, чтобы их тратить. Это понятно. Н..."
1,AIB_0000_0002,AIB_0000,## **Что это за числа и почему вдруг такой рез...,"🔐 **Хранили дома** Несмотря на то, что деньги ..."
2,AIB_0001_0003,AIB_0001,Где и во что можно инвестировать?,"Площадка, где идёт торговля ценными бумагами —..."
3,AIB_0001_0004,AIB_0001,# **Кто торгует на бирже?**,"Продавцами на бирже могут быть сами компании, ..."
4,AIB_0001_0005,AIB_0001,### **Что делает брокер:**,1. Открывает инвесторам [счета](https://alfaba...


## Analyze texts lengths

In [9]:
enc = tiktoken.get_encoding("cl100k_base")

In [39]:
get_tokens_num = lambda text: len(enc.encode(text))

In [40]:
adv_df['num_tokens'] = adv_df.text.apply(get_tokens_num)
beg_df['num_tokens'] = beg_df.text.apply(get_tokens_num)

In [41]:
adv_df.num_tokens.describe(), beg_df.num_tokens.describe()

(count      29.000000
 mean      973.896552
 std       637.543127
 min       187.000000
 25%       577.000000
 50%       797.000000
 75%      1193.000000
 max      3493.000000
 Name: num_tokens, dtype: float64,
 count     38.000000
 mean     280.973684
 std      178.128027
 min       53.000000
 25%      139.000000
 50%      231.000000
 75%      376.000000
 max      768.000000
 Name: num_tokens, dtype: float64)

## Chunkize

### Example

In [108]:
splitter = RecursiveCharacterTextSplitter(chunk_size=256, chunk_overlap=16, length_function=get_tokens_num)

In [109]:
splitter.split_text(adv_df.iloc[5]['text'])

['Облигации можно приобрести напрямую у эмитента при размещении выпуска или у других инвесторов на фондовом рынке, или на внебиржевом рынке.',
 'При покупке облигаций инвестор заплатит за неё номинальную стоимость — если покупает напрямую у эмитента — или биржевую цену, если приобретает её на бирже. Цены облигаций указываются в процентах от номинала, а он всегда составляет 1000 единиц валюты выпуска. Например, цена рублёвой облигации 98,6% означает, что за бумагу нужно заплатить 98,6% от номинала, то есть 986 рублей. Облигации могут выпускаться не только в рублях, эмитент определяет валюту, в которой ему нужны средства. И это могут быть доллары, евро, юани и другие валюты — валютные',
 'юани и другие валюты — валютные облигации традиционно называются еврооблигациями, даже если они в других валютах.',
 'Оценивая выгоды от сделки с облигациями, инвесторы обязательно смотрят на размер купонных выплат по ним, накопленный купонный доход (НКД) и срок погашения — всё это влияет на доходность 

In [110]:
[get_tokens_num(c) for c in splitter.split_text(adv_df.iloc[5]['text'])]

[61, 252, 56, 255, 23, 107]

### Build all chunks

In [111]:
all_chunks = []

In [112]:
for idx, row in tqdm(adv_df.iterrows()):
    chunks = splitter.split_text(row.text)
    for chunk_id, chunk in enumerate(chunks):
        all_chunks.append(
            Document(
                page_content=chunk,
                metadata={
                    "chunk_id" : chunk_id,
                    "paragraph_id" : row.paragraph_id,
                    "article_id" : row.article_id
                }
            )
        )

0it [00:00, ?it/s]

In [113]:
len(all_chunks)

155

In [114]:
for idx, row in beg_df.iterrows():
    chunks = splitter.split_text(row.text)
    for chunk_id, chunk in enumerate(chunks):
        all_chunks.append(
            Document(
                page_content=chunk,
                metadata={
                    "chunk_id" : chunk_id,
                    "paragraph_id" : row.paragraph_id,
                    "article_id" : row.article_id
                }
            )
        )

In [115]:
len(all_chunks)

222

In [116]:
all_chunks[100]

Document(page_content='Если падает цена базового актива, например акций, дешевеет и фьючерс на них. Его владелец несёт убыток из-за списания вариационной маржи и изменения гарантийного обеспечения. При нехватке средств брокер предупредит, что надо пополнить счёт. Если этого не сделать, даже при закрытии позиции баланс счёта может стать отрицательным. Если цена актива резко упала, а на счёте нет денег для пополнения гарантийного обеспечения или образовалась задолженность — брокер имеет право принудительно закрыть позицию по фьючерсу.', metadata={'chunk_id': 5, 'paragraph_id': 'AIA_0004_0016', 'article_id': 'AIA_0004'})

# Vector database

In [96]:
embedding_model = OpenAIEmbeddings(
    model="text-embedding-3-small", 
    openai_api_base = "https://api.vsegpt.ru/v1/"
)

In [92]:
db = FAISS.from_documents(all_chunks, embedding_model)
db.save_local("/Users/alfa/Code/financial_assistant/data/processed/docs_db_index")


## Vector DB search

In [97]:
user_query = """
Ликвидность акции характеризует
А) Способность инвестора продать акцию с минимальными для него потерями в минимальный срок. 
Б) Разницу цены такой акции на разных торговых площадках. 
В) Вероятность погашения акции компанией – эмитентом. 
Г) Ни один из ответов не является правильным.
"""

In [98]:
db.similarity_search(user_query, 3)

[Document(page_content='**Ликвидность** — это экономический термин, показывающий как быстро актив (акции) можно продать по цене, близкой к рыночной, т. е. с минимальными потерями для инвестора. Если спрос на покупку и продажу актива есть всегда и с ним заключается много сделок, такой актив называют высоколиквидным.\n**Волатильность** (англ. volatility) — этот статистический термин характеризует изменчивость показателя в течение определённого времени. Применительно к акциям говорят, что они волатильны, если их цена меняется сильнее и быстрее, чем у большинства других акций.', metadata={'chunk_id': 4, 'paragraph_id': 'AIA_0000_0002', 'article_id': 'AIA_0000'}),
 Document(page_content='-\nАкция, не включённая в котировальный список, как правило, менее ликвидна. А значит, риски не найти продавца или покупателя высоки.\n-\nПри этом отсутствие в котировальном списке вовсе не означает, что цена такой акции будет ниже, чем цены акций из котировального списка.', metadata={'chunk_id': 6, 'paragr

# RAG

In [ ]:
from openai import OpenAI
VSE_GPT_API_KEY = ""

client = OpenAI(
    api_key=VSE_GPT_API_KEY,
    base_url="https://api.vsegpt.ru/v1",
)

## Questions

In [119]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/questions.json', 'r') as f:
    questions = json.load(f)

In [120]:
questions[0]

{'id': '1.4',
 'question': 'Ликвидность акции характеризует',
 'options': [{'letter': 'А',
   'option_text': 'Способность инвестора продать акцию с минимальными для него потерями в минимальный срок.'},
  {'letter': 'Б',
   'option_text': 'Разницу цены такой акции на разных торговых площадках.'},
  {'letter': 'В',
   'option_text': 'Вероятность погашения акции компанией – эмитентом.'},
  {'letter': 'Г',
   'option_text': 'Ни один из ответов не является правильным.'}],
 'answer': 'А',
 'chapter': 1}

In [121]:
with open('/Users/alfa/Code/financial_assistant/data/interim/test_qual_investor/chapters.json', 'r') as f:
    chapters = json.load(f)

In [122]:
chapters

{'1': 'Покупка иностранных акций',
 '2': 'Акции, не включенные в котировальные списки',
 '3': 'Допуск к необеспеченным сделкам (маржинальная торговля)',
 '4': 'Заключение договоров РЕПО',
 '5': 'Опционы, фьючерсы, производные финансовые инструменты',
 '6': 'Структурные облигации',
 '7': 'Паи закрытых паевых инвестиционных фондов (ЗПИФ)',
 '8': 'Облигации российских эмитентов, которым не присвоен рейтинг или он ниже уровня',
 '9': 'Облигации иностранных эмитентов в валюте (еврооблигации) которым не присвоен рейтинг или он ниже нужного уровня',
 '10': 'Облигации со структурным доходом',
 '11': 'Вопросы для допуска к иностранным ETF'}

## Prompt

In [132]:
with open("/Users/alfa/Code/financial_assistant/artifacts/prompts/system_v1.md", "r") as f:
    system_prompt = f.read()

print(system_prompt)

You are qualified financial and investment assistant. 
Provide helpful answers to any question.  
Stricly follow user instructions.  


In [123]:
with open("/Users/alfa/Code/financial_assistant/artifacts/prompts/rag_v1.md", "r") as f:
    prompt_template = f.read()

In [124]:
print(prompt_template)

### Instructions ###

Answer the multiple-choice ###Question### about Russian invest market based on ###Context###.
Follow ###Asnwer Format###.


### Answer Format ###
{{
    "reasoning" : "provide your brief (1-2 sentences) reasoning here",
    "answer": "Б" # one of the first 4 cyrillyc letters: "А", "Б", "В" or "Г"
}}

### Context ###
{context}

### Question ###
{question}

{option_1}
{option_2}
{option_3}
{option_4}




## Evaluation

In [138]:
def get_rag_response(
    question_dict : Dict, 
    prompt_template: str, 
    system_prompt: str, 
    db : FAISS,
    model="openai/gpt-4o-2024-11-20",
    sampling_params: Dict = {
        "max_tokens": 512,
        "temperature": 0.5,
        "top_p": 0.9
    }
):
    
    options = [
        opt["letter"] + '. ' + opt["option_text"]
        for opt in question_dict["options"]
    ]

    question_with_options_text = '\n'.join([question_dict['question']] + options)

    top_docs = db.similarity_search(question_with_options_text, 3)
    top_docs_texts = [f"{i}. {doc.page_content}" for i, doc in enumerate(top_docs)]
    context = '\n'.join(top_docs_texts)

    prompt = prompt_template.format(
        context=context,
        question=question_dict['question'], 
        option_1=options[0],
        option_2=options[1],
        option_3=options[2], 
        option_4=options[3],
    )

    # return prompt

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": "{"}
    ]

    response = client.chat.completions.create(
        model=model,
        messages=messages,
        **sampling_params
    )

    return response

# GPT-4o-mini

In [139]:
results = []
responses = []
for question_dict in tqdm(questions):
    response = get_rag_response(
        question_dict,
        prompt_template,
        system_prompt,
        db,
        model="openai/gpt-4o-mini"
    )
    responses.append(response)
    try:
        response_text =  response.choices[0].message.content
        if '{' not in response_text[:5]:
            response_text = '{' + response_text
        response_dict = json_repair.loads(response_text)

        results.append(
            question_dict | {
                "llm_answer" : response_dict["answer"],
                "llm_reasoning" : response_dict["reasoning"]
            }
        )
    except Exception as e:
        results.append({})

  0%|          | 0/44 [00:00<?, ?it/s]

In [140]:
res_df = pd.DataFrame(results).set_index("id")
res_df['correct'] = (res_df.answer == res_df.llm_answer).astype(int)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1.4 to 11.7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       44 non-null     object
 1   options        44 non-null     object
 2   answer         44 non-null     object
 3   chapter        44 non-null     int64 
 4   llm_answer     44 non-null     object
 5   llm_reasoning  44 non-null     object
 6   correct        44 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ KB


In [141]:
res_df.head()

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
1.4,Ликвидность акции характеризует,"[{'letter': 'А', 'option_text': 'Способность и...",А,1,А,"Ликвидность акции определяет, насколько быстро...",1
1.5,Что из перечисленного не является риском по пр...,"[{'letter': 'А', 'option_text': 'Риск изменени...",А,1,А,Изменение суверенного рейтинга России не влияе...,1
1.6,"В фондовый индекс, рассчитываемый биржей, вклю...","[{'letter': 'А', 'option_text': 'Все акции, до...",Б,1,Б,"Индекс включает акции только тех эмитентов, ко...",1
1.7,"В случае, если Вы купили иностранную акцию за ...","[{'letter': 'А', 'option_text': '500 рублей.'}...",В,1,В,При продаже иностранной акции доход в рублях р...,1
2.4,Вы получили убытки от совершения сделок с акци...,"[{'letter': 'А', 'option_text': 'Нет, не возме...",А,2,А,В контексте торговли акциями с использованием ...,1


In [142]:
res_df.correct.sum(), res_df.correct.mean()

(42, 0.9545454545454546)

In [143]:
res_df.groupby("chapter")['correct'].mean()

chapter
1     1.00
2     1.00
3     1.00
4     1.00
5     1.00
6     1.00
7     1.00
8     1.00
9     1.00
10    0.75
11    0.75
Name: correct, dtype: float64

In [144]:
res_df.to_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o-mini_rag.csv')

# GPT-4o

In [145]:
results = []
responses = []
for question_dict in tqdm(questions):
    response = get_rag_response(
        question_dict,
        prompt_template,
        system_prompt,
        db,
    )
    responses.append(response)
    try:
        response_text =  response.choices[0].message.content
        if '{' not in response_text[:5]:
            response_text = '{' + response_text
        response_dict = json_repair.loads(response_text)

        results.append(
            question_dict | {
                "llm_answer" : response_dict["answer"],
                "llm_reasoning" : response_dict["reasoning"]
            }
        )
    except Exception as e:
        results.append({})

  0%|          | 0/44 [00:00<?, ?it/s]

In [146]:
res_df = pd.DataFrame(results).set_index("id")
res_df['correct'] = (res_df.answer == res_df.llm_answer).astype(int)
res_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44 entries, 1.4 to 11.7
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   question       44 non-null     object
 1   options        44 non-null     object
 2   answer         44 non-null     object
 3   chapter        44 non-null     int64 
 4   llm_answer     44 non-null     object
 5   llm_reasoning  44 non-null     object
 6   correct        44 non-null     int64 
dtypes: int64(2), object(5)
memory usage: 2.8+ KB


In [147]:
res_df.head()

,question,options,answer,chapter,llm_answer,llm_reasoning,correct
id,,,,,,,
1.4,Ликвидность акции характеризует,"[{'letter': 'А', 'option_text': 'Способность и...",А,1,А,Ликвидность акции определяется её способностью...,1
1.5,Что из перечисленного не является риском по пр...,"[{'letter': 'А', 'option_text': 'Риск изменени...",А,1,А,Риск изменения суверенного рейтинга Российской...,1
1.6,"В фондовый индекс, рассчитываемый биржей, вклю...","[{'letter': 'А', 'option_text': 'Все акции, до...",Б,1,Б,"Согласно контексту, акции включаются в фондовы...",1
1.7,"В случае, если Вы купили иностранную акцию за ...","[{'letter': 'А', 'option_text': '500 рублей.'}...",В,1,В,Прибыль от продажи акции в долларах составляет...,1
2.4,Вы получили убытки от совершения сделок с акци...,"[{'letter': 'А', 'option_text': 'Нет, не возме...",А,2,А,"В контексте указано, что убытки от сделок с ак...",1


In [148]:
res_df.correct.sum(), res_df.correct.mean()

(42, 0.9545454545454546)

In [149]:
res_df.groupby("chapter")['correct'].mean()

chapter
1     1.00
2     1.00
3     1.00
4     1.00
5     1.00
6     1.00
7     1.00
8     1.00
9     1.00
10    0.75
11    0.75
Name: correct, dtype: float64

In [150]:
res_df.to_csv('/Users/alfa/Code/financial_assistant/data/results/gpt4o_rag.csv')